In [61]:
import numpy as np
import pandas as pd
from datetime import date
import collections
import datetime
import os
import xarray as xr
import csv

## Generic the filepath to the main data folder
fpath0 = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Parameters/'

scenarios = ['ssp126','ssp245','ssp370','ssp585']

gmodels = ['GloGEM', 'PyGEM', 'OGGM']

parameters = ['Aridity_Index_Avg', 'Precip_Trend_Avg', 'GlacierAreaFrac', 'Mean_Volume_Loss', 'Mean_Initial_Volumes', '#glaciers']

### Reading in:

In [68]:
gmodel_parameters = {}

for gmodel in gmodels:
    # Initialize an empty list to store DataFrames for each parameter and scenario combination
    combined_dfs = []

    for parameter in parameters:
        if parameter in ['Aridity_Index_Avg', 'Precip_Trend_Avg']:
            # Read the CSV for the common parameters (not model-specific)
            fname = f"{parameter}.csv"
            temp_df = pd.read_csv(fpath0 + fname, index_col=0)
            # Rename the column to the parameter name
            temp_df.rename(columns={0: parameter}, inplace=True)
            combined_dfs.append(temp_df)
        elif parameter in ['Mean_Volume_Loss', 'Mean_Initial_Volumes']:
            for SSP in scenarios:
                fname = f"{parameter}_{SSP}.csv"
                temp_df = pd.read_csv(fpath0 + fname, index_col=0)
                # Select the column for the current glacier model
                col_name = f"{parameter}_{SSP}"
                temp_df = temp_df[[gmodel]]
                temp_df.rename(columns={gmodel: col_name}, inplace=True)
                combined_dfs.append(temp_df)
        else:
            fname = f"{parameter}.csv"
            temp_df = pd.read_csv(fpath0 + fname, index_col=0)
            # Select the column for the current glacier model
            col_name = f"{parameter}"
            temp_df = temp_df[[gmodel]]
            temp_df.rename(columns={gmodel: col_name}, inplace=True)
            combined_dfs.append(temp_df)
    
    # Concatenate all the DataFrames in the list
    combined_df = pd.concat(combined_dfs, axis=1)

    # Set a title above the column indexes
    combined_df.columns.name = gmodel

    # Store the combined_df in the dictionary using the glacier model as the key
    gmodel_parameters[gmodel] = combined_df

    # Rename the aridity_index column name 
    gmodel_parameters[gmodel].columns = gmodel_parameters[gmodel].columns.map(lambda x: x if x != '0' else 'Aridity_Index_Avg')


In [67]:
#Saving dfs as CSVs
output_dir = '/Users/finnwimberly/Desktop/Lizz Research/CSV Outputs/Parameters/MASTERS/'

for gmodel in gmodels:
    fname = f"MASTER_{gmodel}.csv"

    # Define the full path of the output file
    output_path = os.path.join(output_dir, fname)

    # Save the DataFrame as CSV
    gmodel_parameters[gmodel].to_csv(output_path, header=True, index=True)